# 9. Deep Learning with PyTorch

이번 세션에서는 PyTorch 패키지를 사용하여 ML모델의 정의, 학습 및 예측 실습을 합니다.

먼저 다음의 코드를 실행하여 필요한 라이브러리들을 import 합니다.


In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import time

## 9.1. Getting Started with PyTorch

먼저 PyTorch이 제공하는 유용한 기능들을 unit 별로 실습해보는 시간을 갖습니다.

(Tutorial 출처: https://github.com/yunjey/pytorch-tutorial) 


### 9.1.1. Autograd

자동으로 미분을 해주는 기능인 autograd(automatic differentiation, AD)을 사용해 봅니다.

In [ ]:
# 먼저 tensor 타입의 변수를 선언하고, 계산 그래프를 생성합니다
x = torch.tensor(1., requires_grad=True)
w = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)

y = w * x + b    # y = 2 * x + 3

# backward() 함수는 계산 그래프로부터 미분값을 계산합니다.
y.backward()


이제 자동으로 미분된 결과값을 출력해볼까요?

In [ ]:
print(x.grad)    # x.grad = 2 
print(w.grad)    # w.grad = 1 
print(b.grad)    # b.grad = 1 


실제 모델 학습 과정에서 autograd이 사용되는 모습은 다음과 같습니다 (다음 코드는 실습해보지 않아도 괜찮습니다).

In [ ]:
# Random한 값으로 데이터를 만듭니다. x의 사이즈는 10x3; y의 사이즈는 10x2 입니다
x = torch.randn(10, 3)
y = torch.randn(10, 2)

# x를 입력, y를 출력으로 다룰 수 있는 Linear regression model을 만듭니다
# (nn 표현으로는 fully connected layer를 생성합니다)
linear = nn.Linear(3, 2)
print ('w: ', linear.weight)
print ('b: ', linear.bias)

# 학습에 사용될 loss function 및 optimizer를 선언 합니다
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

# Forward pass
pred = linear(x)

# Compute loss
loss = criterion(pred, y)
print('** loss before 1 step optimization: ', loss.item())

# Backward pass
loss.backward()

# 계산된 미분값을 출력합니다
print ('dL/dw: ', linear.weight.grad) 
print ('dL/db: ', linear.bias.grad)

# gradient descent의 1 iteration만 실행해 봅니다.
optimizer.step()

# You can also perform gradient descent at the low level
# linear.weight.data.sub_(0.01 * linear.weight.grad.data)
# linear.bias.data.sub_(0.01 * linear.bias.grad.data)

# gradient descent의 1 iteration 후, 업데이트된 loss 값을 출력합니다.
pred = linear(x)
loss = criterion(pred, y)
print('** loss after 1 step optimization: ', loss.item())


### 9.1.2. Loading data from `numpy`

`torch.from_numpy()` 함수를 사용하면 `ndarray` 타입(`numpy`의 다차원 행렬 타입)의 데이터를 torch에서 사용하는 데이터 타입으로 (and *vice versa*) 쉽게 변환 가능합니다.

In [ ]:
import numpy as np

# numpy array를 생성
x = np.array([[1, 2], [3, 4]])

# numpy array -> torch tensor
y = torch.from_numpy(x)

# torch tensor -> numpy array
z = y.numpy()

### 9.1.3. DataLoader

`DataLoader`는 학습 및 예측 과정에서 모델에 데이터를 공급해주는 역할을 합니다.

`DataLoader`는 `sklearn.datasets`와 같이 자주 쓰이는 데이터를 내장하고 있으며, 다음 예제를 통해 CIFAR-10이라는 이미지 데이터를 불러오는 작업을 수행해볼 수 있습니다. 

In [ ]:
# CIFAR-10 데이터셋을 다운로드 및 메모리에 로드
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True, 
                                             transform=transforms.ToTensor(),
                                             download=True)

# 데이터셋의 첫 번째 instance로 접근해보기
image, label = train_dataset[0]
print (image.size())
print (label)

# 데이터셋으로부터 DataLoader 생성
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=64, 
                                           shuffle=True)

# DataLoader로부터 데이터를 받으며 학습하는 code skeleton
for images, labels in train_loader:
    # ----------------------------------------------
    # -- Your training code should be placed here --
    # ----------------------------------------------
    pass

# Iterator를 사용하여 미니배치(mini-batch)를 구현할 수도 있음
# data_iter = iter(train_loader)
# images, labels = data_iter.next()


다음의 code skeleton을 사용하면, 개별 데이터셋을 torch에서 사용되는 데이터셋 타입으로 불러들일 수 있습니다. 이렇게 작성된 dataset은 DataLoader와 함께 사용될 수 있습니다.

In [ ]:
# 다음 code skeleton을 사용하여 DataLoader를 구성할 수 있습니다
# (TODO에 해당하는 내용을 채우기 전에는 실행되지 않습니다)

# You should build your custom dataset as below
class CustomDataset(torch.utils.data.Dataset):
  def __init__(self):
    # TODO
    # 1. Initialize file paths or a list of file names
    pass
  def __getitem__(self, index):
    # TODO
    # 1. Read one data from file (e.g. using numpy.fromfile, PIL.Image.open)
    # 2. Preprocess the data (e.g. torchvision.Transform)
    # 3. Return a data pair (e.g. image and label)
    pass
  def __len__(self):
    # You should change 0 to the total size of your dataset
    return 0 

# You can then use the prebuilt data loader
custom_dataset = CustomDataset()
train_loader = torch.utils.data.DataLoader(dataset=custom_dataset,
                                           batch_size=64, 
                                           shuffle=True)

### 9.1.4. Loading a Pretrained Model

PyTorch를 통해 작성되고, 학습된 모델(pretrained model)을 파일 형태로 주고 받을 수도 있습니다. 다음 코드는 PyTorch를 통해 제공되는 pretrained ResNet-18 (이미지 분류 모델)을 불러옵니다.

In [ ]:
# Pretrained ResNet-18 다운로드 및 불러오기
resnet = torchvision.models.resnet18(pretrained=True)


# 다운로드 받은 pretrained model을 추가로 더 학습하는 것도 가능합니다
# 다음 코드는 ResNet-18의 가장 마지말 레이어만 추가 학습하는 예시를 보여줍니다
for param in resnet.parameters():
    param.requires_grad = False
resnet.fc = nn.Linear(resnet.fc.in_features, 100)  # 100 is an example


# Forward pass를 통해 pretrained model을 예측 작업에 바로 적용해볼 수 있습니다
images = torch.randn(64, 3, 224, 224)
outputs = resnet(images)
print (outputs.size())     # (64, 100)

## 9.2. Logistic Regression (revisited)

이번 절에서는 우리가 이미 알고있는 logistic regression 모델을 PyTorch로 구현해보는 과정을 통해, scikit-learn (sklearn)과는 다른 PyTorch 용법을 실습해 봅니다.

실습에 사용할 데이터를 불러와 DataLoader를 구성합니다. 사용할 데이터는 손글씨 숫자 이미지를 담고있는 MNIST 데이터셋 입니다. 

![Image is not found](https://miro.medium.com/max/530/1*VAjYygFUinnygIx9eVCrQQ.png)

In [2]:
input_size = 784
num_classes = 10
batch_size = 100

# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

URLError: <urlopen error [Errno -3] Temporary failure in name resolution>

PyTorch로 logistic regression 모델을 정의하는 과정은 다음과 같습니다.

In [3]:
# 실험에 사용할 하이퍼파라메터들을 설정 합니다
learning_rate = 0.001

# 하나의 linear 레이어로 구성된 네트워크를 만듭니다
model = nn.Linear(input_size, num_classes)

# 모델이 사용할 loss function(i.e., objective function)과 optimizer를 지정합니다
# nn.CrossEntropyLoss() computes softmax internally
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


위 모델을 위한 학습 및 예측 함수를 정의합니다.

In [4]:
# Train the model
def train_logreg(train_loader, num_epochs):
  total_step = len(train_loader)
  for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
      # Reshape images to (batch_size, input_size)
      images = images.reshape(-1, 28*28)
      
      # Forward pass
      outputs = model(images)
      loss = criterion(outputs, labels)
      
      # Backward and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      
      # Display the progress
      if (i+1) % 300 == 0:
        print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
              .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
        
# Test the model
def test_logreg(model, test_loader):
  # In test phase, we don't need to compute gradients (for memory efficiency)
  with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
      images = images.reshape(-1, 28*28)
      outputs = model(images)
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum()

    # Display the result
    print('Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

위에서 정의한 함수들을 사용하여 logistic regression 모델을 학습/평가해 봅니다. 

얼마나 높은 accuracy를 얻을 수 있나요?

In [5]:
tr_start=time.time()
train_logreg(train_loader, num_epochs=10)
print("train time: ", time.time()-tr_start)

ts_start=time.time()
test_logreg(model, test_loader)
print("test time: ", time.time()-ts_start)

NameError: name 'time' is not defined

## 9.3. Feedforward Neural Networks

이번 절에서는 가장 기초적인 neural network이라고 할 수 있는 Feedforward Neural Network (multi-layer perceptron, MLP)을 작성하고 MNIST 데이터를 학습하는 데에 적용해보겠습니다.

먼저, 실험에 사용할 하이퍼파라메터들을 설정 합니다.

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters 
hidden_size = 500
learning_rate = 0.001

다음 class definition을 통해, 이번 절에서 사용할 FFNet 모델을 정의합니다.

In [ ]:
# Fully connected neural network with one hidden layer
class FFNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(FFNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out


위 모델을 위한 학습 및 예측 함수를 정의합니다.

In [ ]:
# Train the model
def train_ffnet(model, train_loader, num_epochs):
  total_step = len(train_loader)
  for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
      # Move tensors to the configured device
      images = images.reshape(-1, 28*28).to(device)
      labels = labels.to(device)
      
      # Forward pass
      outputs = model(images)
      loss = criterion(outputs, labels)
      
      # Backward and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      
      # Display the progress
      if (i+1) % 300 == 0:
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
               .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
        
# Test the model
def test_ffnet(model, test_loader):
  # In test phase, we don't need to compute gradients (for memory efficiency)
  with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
      images = images.reshape(-1, 28*28).to(device)
      labels = labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()

    # Display the result
    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))


위에서 정의한 모델과 함수들을 사용하여 MNIST 데이터에 적용해 봅니다. 

FFNet을 사용하여 얼마나 높은 accuracy를 얻을 수 있나요?

In [ ]:
# declare a model object (instantiation)
model = FFNet(input_size, hidden_size, num_classes).to(device)

# Set the loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

train_ffnet(model, train_loader, num_epochs=10)
test_ffnet(model, test_loader)

## 9.4. Convolutional Neural Networks

이번 절에서는 이미지 학습/예측에 적합한 neural network의 한 종류인 Convolutional Neural Network (CNN)을 작성하고 MNIST 데이터에 적용해보겠습니다.

가장 먼저, 실험에 사용할 하이퍼파라메터들을 설정 합니다.

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters 
learning_rate = 0.001

다음 class definition을 통해, 이번 절에서 사용할 CNN 모델을 정의합니다.

In [ ]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
  def __init__(self, num_classes=10):
    super(ConvNet, self).__init__()
    self.layer1 = nn.Sequential(
      nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
      nn.BatchNorm2d(16),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2))
    self.layer2 = nn.Sequential(
      nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
      nn.BatchNorm2d(32),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2))
    self.fc = nn.Linear(7*7*32, num_classes)
      
  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = out.reshape(out.size(0), -1)
    out = self.fc(out)
    return out

위 모델을 위한 학습 및 예측 함수를 정의합니다.

In [ ]:
# Train the model
def train_convnet(train_loader, num_epochs):
  total_step = len(train_loader)
  for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
      images = images.to(device)
      labels = labels.to(device)
      
      # Forward pass
      outputs = model(images)
      loss = criterion(outputs, labels)
      
      # Backward and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      
      # Display the progress
      if (i+1) % 300 == 0:
        print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
              .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# Test the model      
def test_convnet(model, test_loader):
  model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
  with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
      images = images.to(device)
      labels = labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()

    # Display the result
    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

위에서 정의한 모델과 함수들을 사용하여 MNIST 데이터에 적용해 봅니다. 

CNN을 사용하여 얼마나 높은 accuracy를 얻을 수 있나요?

In [ ]:
model = ConvNet(num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

tr_start=time.time()
train_convnet(train_loader, num_epochs=10)
print("train time: ", time.time()-tr_start)

ts_start=time.time()
test_convnet(model, test_loader)
print("test time: ", time.time()-ts_start)

## 9.5. Skorch (Optional)

Skorch는 PyTorch를 scikit-learn (sklearn) 라이브러리와 함께 사용할 수 있게 해주는 wrapper 라이브러리 입니다. Skorch를 사용하면, sklearn에서 유용하게 사용되는 pipeline 구조(이번 캠프에서 다루지는 않았습니다)와 GridSearchCV를 PyTorch와도 사용할 수 있게 됩니다.

먼저 해당 라이브러리를 설치 합니다.

In [ ]:
!pip install skorch

이제 sklearn이 처리할 수 있는 타입으로 데이터셋을 다시 준비합니다.

In [ ]:
# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=len(train_dataset), 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=len(test_dataset), 
                                          shuffle=False)

# numpy array
X_tr = next(iter(train_loader))[0].numpy()
y_tr = next(iter(train_loader))[1].numpy()
X_ts = next(iter(test_loader))[0].numpy()
y_ts = next(iter(test_loader))[1].numpy()

X_tr = X_tr.reshape(-1, input_size)
X_ts = X_ts.reshape(-1, input_size)

X_ts = X_ts / X_tr.max()
X_tr = X_tr / X_tr.max()
print(X_tr.shape)
print(y_tr.shape)
print(X_ts.shape)
print(y_ts.shape)

이제 GridSearchCV framework을 통해, FFNet 모델을 학습해 봅니다. 즉, 다음 코드를 통해 다음 하이퍼파라메터 중 최적의 조합을 찾습니다.
 - 'lr': [0.01, 0.001, 0.0001]
 - 'max_epochs': [5, 10, 15]
 

In [ ]:
from skorch import NeuralNetClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# from sklearn.metrics import make_scorer
# from skorch.callbacks import EpochScoring

# def accuracy_argmax(y_true, y_pred):
#   return np.mean(y_true == np.argmax(y_pred, -1))

# accuracy_argmax_scorer = make_scorer(accuracy_argmax)

# epoch_acc = EpochScoring(
#     accuracy_argmax_scorer,
#     name='valid_acc',
#     lower_is_better=False
# )

# hyper-parameters
input_size = 784
num_classes = 10
batch_size = 100
hidden_size = 500

net = NeuralNetClassifier(FFNet, 
                          batch_size = 100,
                          criterion = nn.CrossEntropyLoss,
                          optimizer = torch.optim.Adam,
                          # callbacks=[epoch_acc],
                          iterator_train__shuffle=True)

params = {
    'lr': [0.001], #[0.01, 0.001, 0.0001],
    'max_epochs': [10], #[5, 10, 15],
    'module__input_size': [input_size],
    'module__hidden_size': [hidden_size],
    'module__num_classes': [num_classes]
}
gridsearch = GridSearchCV(net, params, cv=3, scoring='accuracy')


gridsearch.fit(X_tr, y_tr)
print(gridsearch.best_score_, gridsearch.best_params_)

best_net = gridsearch.best_estimator_
y_pred = best_net.predict(X_ts)


In [ ]:
print(y_pred[0:10])
print(y_ts[0:10])


## References
* yunjey. PyTorch Tutorial for Deep Learning Researchers. URL: https://github.com/yunjey/pytorch-tutorial
* Skorch Tutorials. URL: https://skorch.readthedocs.io/en/stable/user/tutorials.html